In [10]:
import numpy as np
from PuzzleLib.Backend import gpuarray
from PuzzleLib.Modules import Conv2D, BatchNorm2D, Activation, Concat, relu, AvgPool2D, Linear, Reshape
from PuzzleLib.Variable import Variable
from PuzzleLib.Containers import Graph, Sequential

import os
import math

import numpy as np

from PuzzleLib.Datasets import Cifar10Loader

from PuzzleLib.Containers import Sequential
from PuzzleLib.Modules import Conv2D, MaxPool2D, Activation, Flatten, Linear
from PuzzleLib.Modules.Activation import relu
from PuzzleLib.Handlers import Trainer, Validator
from PuzzleLib.Optimizers import MomentumSGD
from PuzzleLib.Cost import CrossEntropy

from PuzzleLib.Visual import showImageBasedFilters, showFilters

In [13]:
import math

def BasicBlock(inmaps, outsize):
    bn = BatchNorm2D(inmaps, name="bn").node()
    rel = Activation(relu, name="act").node(bn)
    conv =  Conv2D(inmaps=inmaps, name="conv", 
                            outmaps=outsize, size=3, stride=1, pad=1, useBias=False).node(rel)
    concat = Concat(axis=1, name="concat").node(conv, bn)
    graph = Graph(inputs=bn, outputs=concat)
    return graph
    

    
def BottleneckBlock(inmaps, outsize):
        inter = outsize * 4
        bn1 = BatchNorm2D(inmaps, name="bn1").node()
        relu1 = Activation(relu, name="act1").node(bn1)
        conv1 = Conv2D(inmaps=inmaps, name="conv1", 
                            outmaps=inter, size=1, stride=1, pad=0, useBias=False).node(relu1)
        bn2 = BatchNorm2D(inter, name="bn2").node(conv1)
        relu2 = Activation(relu, name="act2").node(bn2)
        conv2 = Conv2D(inter, outsize, size=3, stride=1,
                               pad=1, useBias=False).node(relu2)
        concat = Concat(axis=1, name="concat").node(bn1, conv2)
        graph = Graph(inputs=bn1, outputs=concat)
        return graph
    
    
def TransitionBlock(inmaps, outsize):
        bn = BatchNorm2D(inmaps, name="bn").node()
        rel = Activation(relu, name="act").node(bn)
        conv = Conv2D(inmaps, outsize, size=1, stride=1,
                               pad=0, useBias=False).node(rel)
        pool = AvgPool2D(name="pool").node(conv)
        graph = Graph(inputs=bn, outputs=pool)
        return graph
    
def DenseBlock(num_layers, inmaps, growth_rate, block):
    layers = Sequential()
    for i in range(num_layers):
        layers.append(block(inmaps + i * growth_rate, growth_rate))
    return layers

def DenseNet3(depth, num_classes, growth_rate=12,
                 reduction=0.5, bottleneck=True):
    layers = Sequential()
    imaps = 2 * growth_rate
    n = (depth - 4) // 3
    if bottleneck == True:
        n = n // 2
        block = BottleneckBlock
    else:
        block = BasicBlock
    
    layers.append(Conv2D(3, imaps, size=3, stride=1,
                               pad=1, useBias=False))
    layers.append(DenseBlock(n, imaps, growth_rate, block))
    imaps = int(imaps + n * growth_rate)
    layers.append(TransitionBlock(imaps, int(math.floor(imaps * reduction))))
    imaps = int(math.floor(imaps * reduction))
    layers.append(DenseBlock(n, imaps, growth_rate, block))
    imaps = int(imaps + n * growth_rate)
    layers.append(TransitionBlock(imaps, int(math.floor(imaps * reduction))))
    imaps = int(math.floor(imaps * reduction))
    layers.append(DenseBlock(n, imaps, growth_rate, block))
    imaps = int(imaps + n * growth_rate)
    layers.append(BatchNorm2D(imaps))
    layers.append(Activation(relu))
    layers.append(AvgPool2D(name="pool"))
    layers.append(Flatten())
    layers.append(Linear(layers.dataShapeFrom((1, 3, 32, 32))[1], 64, wscale=0.1, initscheme="gaussian"))
    layers.append(Activation(relu))
    layers.append(Linear(64, num_classes, wscale=0.1, initscheme="gaussian"))
    return layers

In [14]:
b = DenseNet3(20, 10)

In [15]:
data = gpuarray.to_gpu(np.ones((32, 3, 32, 32)).astype(np.float32))

In [16]:
data.shape

(32, 3, 32, 32)

In [17]:
b(data).shape

(32, 10)

In [18]:
!cd TestData/ && wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz -O cifar-10-python.tar.gz

--2021-05-26 20:34:35--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: 'cifar-10-python.tar.gz'

cifar-10-python.tar 100%[===================>] 162.60M  11.6MB/s    in 16s     

2021-05-26 20:34:52 (10.1 MB/s) - 'cifar-10-python.tar.gz' saved [170498071/170498071]



In [22]:
cifar10 = Cifar10Loader()
path = "./TestData/"
data, labels = cifar10.load(path=path)
data, labels = data[:], labels[:]
print("Loaded cifar10")

np.random.seed(1234)

Loaded cifar10


In [23]:
optimizer = MomentumSGD()
optimizer.setupOn(b, useGlobalState=True)
optimizer.learnRate = 0.01
optimizer.momRate = 0.9

cost = CrossEntropy(maxlabels=10)

trainer = Trainer(b, cost, optimizer)

validator = Validator(b, cost)

In [24]:
currerror = math.inf

In [25]:
labels[:100].shape

(100,)

In [26]:
for i in range(25):
    trainer.trainFromHost(
    data[:50000], labels[:50000], macroBatchSize=50000,
    onMacroBatchFinish=lambda train: print("Train error: %s" % train.cost.getMeanError())
    )
    valerror = validator.validateFromHost(data[50000:], labels[50000:], macroBatchSize=10000)
    Accuracy = 1.0 - valerror
    print("Accuracy:", Accuracy)
    
    if valerror >= currerror:
        optimizer.learnRate *= 0.5
        print("Lowered learn rate: %s" % optimizer.learnRate)

    currerror = valerror

Train error: 1.46144
Accuracy: 0.523
Train error: 1.094353984375
Accuracy: 0.6194
Train error: 0.936843203125
Accuracy: 0.6384000000000001
Train error: 0.827614765625
Accuracy: 0.6759999999999999
Train error: 0.75216296875
Accuracy: 0.6946
Train error: 0.697643515625
Accuracy: 0.7174
Train error: 0.646631015625
Accuracy: 0.7215
Train error: 0.6072823828125
Accuracy: 0.7194
Lowered learn rate: 0.005
Train error: 0.523757734375
Accuracy: 0.75
Train error: 0.49664015625
Accuracy: 0.7601
Train error: 0.4779570703125
Accuracy: 0.7499
Lowered learn rate: 0.0025
Train error: 0.423093046875
Accuracy: 0.7655000000000001
Train error: 0.4057825390625
Accuracy: 0.7633
Lowered learn rate: 0.00125
Train error: 0.3757782421875
Accuracy: 0.7709
Train error: 0.3647407421875
Accuracy: 0.7696000000000001
Lowered learn rate: 0.000625
Train error: 0.347511953125
Accuracy: 0.7705
Train error: 0.34225375
Accuracy: 0.7703
Lowered learn rate: 0.0003125
Train error: 0.33272578125
Accuracy: 0.7723
Train error: 0

In [27]:
print("Final accuracy:", Accuracy)

Final accuracy: 0.7738
